# Demo Tugas Akhir 2
---
Berikut adalah demo implementasi solusi Tugas Akhir 2 Pengujian Adversarial Robustness pada Model Bahasa Pra-latih dalam Domain Klasifikasi Teks. Dalam demo ini seluruh komponen dalam rancangan solusi telah diimplementasikan pada package yang diimport pada cell pertama.

In [ ]:
import os, sys
import gc

gc.collect()
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

from sklearn.metrics import accuracy_score
import swifter
import torch
device = torch.device("cuda")

from tqdm.notebook import tqdm
tqdm.pandas()

from utils.utils_init_dataset import set_seed, load_dataset_loader
from utils.utils_semantic_use import USE
from utils.utils_data_utils import DocumentSentimentDataset, DocumentSentimentDataLoader, EmotionDetectionDataset, EmotionDetectionDataLoader
from utils.utils_metrics import document_sentiment_metrics_fn
from utils.utils_init_model import text_logit, fine_tuning_model, eval_model, init_model, logit_prob, load_word_index
from utils.get_args import get_args

from attack.adv_attack import attack
from attack.attack_helper import read_dict
import os, sys

import pandas as pd
pd.set_option('display.max_colwidth', None)

### Deklarasi variabel untuk pengujian
---
Pada pengujian kali ini dilakukan terhadap model IndoBERT-Base untuk task analisis sentimen dengan code-mixing Bahasa Perancis.

In [50]:
model_target = "IndoBERT"
downstream_task = "sentiment"
attack_strategy = "adversarial"
finetune_epoch = 10
num_sample = 10
exp_name = "demo"
perturbation_technique = "codemixing"
perturb_ratio = 0.8
dataset = "valid"
perturb_lang="it"
seed=26092020

### Inisialisasi Pengujian
---
Pada tahap ini diinisialisasi random seed yang digunakan, kemudian inisialisasi model Universal Sentence Encoder, inisialisasi model, dan inisialisasi dataset.

In [51]:
set_seed(seed)
use = USE()

tokenizer, config, finetuned_model = init_model(model_target, downstream_task, seed)
w2i, i2w = load_word_index(downstream_task)

train_dataset, train_loader, train_path = load_dataset_loader(downstream_task, 'train', tokenizer)
valid_dataset, valid_loader, valid_path = load_dataset_loader(downstream_task, 'valid', tokenizer)
test_dataset, test_loader, test_path = load_dataset_loader(downstream_task, 'test', tokenizer)

finetuned_model.to(device)


translator = None
if perturbation_technique == "codemixing":
    translator = read_dict(os.getcwd() + r"/dicts/new_dict_"+perturb_lang+".txt")

In [44]:
translator['protes']

'protes'

### Tahap Pengujian
--- 
Dilakukan pengujian dengan parameter yang telah didefinisikan sebelumnya. Pengujian dilakukan secara iteratif pada dataset validasi menggunakan fungsi adversarial attack yang telah diimport pada package diatas.

In [52]:
if dataset == "valid":
    exp_dataset = valid_dataset.load_dataset(valid_path).sample(num_sample)
elif dataset == "train":
    exp_dataset = train_dataset.load_dataset(train_path)
    
text,label = None,None

if downstream_task == 'sentiment':
        text = 'text'
        label = 'sentiment'
        exp_dataset[['perturbed_text', 'perturbed_semantic_sim', 'pred_label', 'pred_proba', 'perturbed_label', 'perturbed_proba', 'translated_word(s)', 'running_time(s)']] = exp_dataset.progress_apply(
            lambda row: attack(
                text_ls = row.text,
                true_label = row.sentiment,
                predictor = finetuned_model,
                tokenizer = tokenizer, 
                att_ratio = perturb_ratio,
                perturbation_technique = perturbation_technique,
                translator=translator,
                lang_codemix = perturb_lang,
                sim_predictor = use), axis=1, result_type='expand'
        )
elif downstream_task == 'emotion':
    text = 'tweet'
    label = 'label'
    exp_dataset[['perturbed_text', 'perturbed_semantic_sim', 'pred_label', 'pred_proba', 'perturbed_label', 'perturbed_proba', 'translated_word(s)', 'running_time(s)']] = exp_dataset.progress_apply(
        lambda row: attack(
            text_ls = row.tweet,
            true_label = row.label,
            predictor = finetuned_model,
            tokenizer = tokenizer, 
            att_ratio = perturb_ratio,
            perturbation_technique = perturbation_technique,
            translator=translator,
            lang_codemix = perturb_lang,
            sim_predictor = use), axis=1, result_type='expand'
    )


  0%|          | 0/10 [00:00<?, ?it/s]

### Tahap Evaluasi
---
Pada tahap ini dilakukan evaluasi dari adversarial examples dan label prediksi yang telah dihasilkan pada cell sebelumnya. Pengujian dilakukan dengan melihat delta akurasi dan skor kemiripan semantik.

In [53]:
before_attack = accuracy_score(exp_dataset[label], exp_dataset['pred_label'])
after_attack = accuracy_score(exp_dataset[label], exp_dataset['perturbed_label'])

exp_dataset.loc[exp_dataset.index[0], 'before_attack_acc'] = before_attack
exp_dataset.loc[exp_dataset.index[0], 'after_attack_acc'] = after_attack
exp_dataset.loc[exp_dataset.index[0], 'delta_acc'] = before_attack-after_attack
exp_dataset.loc[exp_dataset.index[0], 'avg_semantic_sim'] = exp_dataset["perturbed_semantic_sim"].mean()
exp_dataset.loc[exp_dataset.index[0], 'avg_running_time(s)'] = exp_dataset["running_time(s)"].mean()
# exp_dataset.to_csv(os.getcwd() + r'/result/seed'+str(seed)+"/"+str(dataset)+"/"+str(exp_name)+".csv", index=False)

In [54]:
exp_dataset

,text,sentiment,perturbed_text,perturbed_semantic_sim,pred_label,pred_proba,perturbed_label,perturbed_proba,translated_word(s),running_time(s),before_attack_acc,after_attack_acc,delta_acc,avg_semantic_sim,avg_running_time(s)
799,"ayo kita bantu viralkan gerakan petani tebu yang tidak diliput luas oleh media , kita protes kebijakan pemerintahan jokowi yang kata nya pro rakyat kecil , ternyata pro wong licik . ternyata kita dipimpin rezim drakula impor",2,Dai kita auto viralkan gerakan contadino tebu yang tidak diliput luas oleh media kita protes kebijakan pemerintahan jokowi yang kata nya pro rakyat kecil ternyata pro wong licik ternyata kita portato rezim drakula impor,0.960896,2,"[0.0024, 0.002, 0.9956]",2,"[0.0033, 0.003, 0.9936]","{'protes': 'protes', 'licik': 'licik', 'impor': 'impor', 'jokowi': 'jokowi', 'media': 'media', 'viralkan': 'viralkan', 'rezim': 'rezim', 'pemerintahan': 'pemerintahan', 'ayo': 'Dai', 'tebu': 'tebu', 'bantu': 'auto', 'rakyat': 'rakyat', 'petani': 'contadino', 'pro': 'pro', 'diliput': 'diliput', 'dipimpin': 'portato', 'nya': 'nya'}",0.4681,0.9,0.9,0.0,0.945742,0.14008
163,"tempat nya oke , standarlah seperti mcdonalds lainnya . sekarang lebih sering pesan lewat drive thru , karena irit waktu lebih efisien jadi nya",0,tempat nya oke standarlah seperti mcdonalds lainnya sekarang lebih sering pesan lewat drive thru karena irit waktu lebih efisien jadi nya,1.000000,0,"[0.9957, 0.002, 0.0023]",0,"[0.9928, 0.0027, 0.0045]","{'mcdonalds': 'mcdonalds', 'irit': 'irit', 'thru': 'thru', 'nya': 'nya', 'efisien': 'efisien', 'drive': 'drive', 'pesan': 'pesan'}",0.0839,NaN,NaN,NaN,NaN,NaN
906,asik . lagu lama . menyelundupkan saja berani tertangkap nangis kayak bocah . hukum mati saja biar kapok,2,Assorbito lagu lama menyelundupkan saja berani tertangkap piangere kayak bocah hukum mati saja biar kapok,0.963167,2,"[0.0008, 0.0015, 0.9977]",2,"[0.0006, 0.0023, 0.9972]","{'kapok': 'kapok', 'asik': 'Assorbito', 'hukum': 'hukum', 'mati': 'mati', 'nangis': 'piangere', 'tertangkap': 'tertangkap', 'kayak': 'kayak', 'bocah': 'bocah', 'menyelundupkan': 'menyelundupkan'}",0.0698,NaN,NaN,NaN,NaN,NaN
1244,"buset lama amat , jangan bilang max 1x24 jam dong kalau lama",2,buset lama amat jangan bilang max 1x24 minerale dong kalau lama,0.946869,2,"[0.0007, 0.0016, 0.9977]",2,"[0.0007, 0.0019, 0.9974]","{'buset': 'buset', 'jam': 'minerale', 'max': 'max'}",0.0635,NaN,NaN,NaN,NaN,NaN
717,saya tidak percaya negara agama . agama tidak perlu negara . tuhan tak perlu kamu !,2,saya tidak ritenere nazione religione religione tidak perlu nazione tuhan tak perlu kamu,0.897557,2,"[0.003, 0.002, 0.9951]",2,"[0.0899, 0.0479, 0.8622]","{'percaya': 'ritenere', 'negara': 'nazione', 'agama': 'religione'}",0.0646,NaN,NaN,NaN,NaN,NaN
320,terong yang digoreng dengan tepung dan juga maskan khas sunda lainnya menurut saya sangat lezat rasanya ditambah pelayanan dari karyawan nya yang cukup baik karena pas saya datang ke sana tempat nya cukup ramai sehingga menunggu ada tempat kosong . cuma untuk yang bawa kendaraan akan cukup sulit mendapat tempat parkir di saat tempat nya ramai apalagi di akhir pekan . tetapi berlebihan ala saya suka dan puas .,0,melanzana yang fritto dengan tepung dan juga maskan khas Sonda lainnya menurut saya sangat lezat rasanya ditambah pelayanan dari karyawan nya yang cukup baik karena pas saya datang ke sana tempat nya cukup affolato sehingga aspettare ada tempat kosong cuma untuk yang portare kendaraan akan cukup sulit mendapat tempat parcheggio di saat tempat nya affolato apalagi di akhir pekan tetapi berlebihan ala saya suka dan puas,0.976579,0,"[0.9976, 0.0008, 0.0016]",0,"[0.9975, 0.0009, 0.0015]","{'suka': 'suka', 'puas': 'puas', 'maskan': 'maskan', 'pekan': 'pekan', 'ala': 'ala', 'ramai': 'affolato', 'bawa': 'portare', 'lezat': 'lezat', 'tepung': 'tepung', 'parkir': 'parcheggio', 'kendaraan': 'kendaraan', 'khas': 'khas', 'terong': 'melanzana', 'sunda': 'Sonda', 'menunggu': 'aspettare', 'pas': 'pas', 'pelayanan': 'pelayana